In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import googlemaps
import gmaps
import os
import requests
import shutil
from google.cloud import translate
from IPython.display import Image

In [13]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [10]:
input_folder = '../data/railways/China_HSR_2016_lines/'
input_file = input_folder+'China_HSR_2016_lines.shp'

output_folder = '../data/output_images/China/'
output_csv = output_folder+'CHN.csv'

#### Import routes

In [11]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4326'})

routes.head()

,OBJECTID,line_chin,line_py,condition,speed_txt,kph,geometry
0,1,京石动车组,jīng shí dòng chē zǔ,Upgrade,200-250km/hr,200,LINESTRING Z (115.637697612 39.03169797600003 ...
1,2,杭甬线,háng yǒng xiàn,Existing,300+km/hr,300,LINESTRING Z (120.208261468 30.29341594800007 ...
2,3,宁杭客运专线,níng háng kè yùn zhuān xiàn,Existing,300+km/hr,300,(LINESTRING Z (120.18048509 30.33301164900007 ...
3,4,汉宜,hàn yí,New,200-250km/hr,200,LINESTRING Z (111.3649862190001 30.66111361200...
4,5,京港通道,jīng gǎng tōng dào,Existing,300+km/hr,300,(LINESTRING Z (113.2928800510001 22.9317548520...


In [12]:
routes.shape

(79, 7)

In [14]:
routes['kph'].value_counts()

200    38
250    25
300    16
Name: kph, dtype: int64

In [19]:
routes = routes[routes['kph']==300]
routes.head()

,OBJECTID,line_chin,line_py,condition,speed_txt,kph,geometry
1,2,杭甬线,háng yǒng xiàn,Existing,300+km/hr,300,LINESTRING Z (120.208261468 30.29341594800007 ...
2,3,宁杭客运专线,níng háng kè yùn zhuān xiàn,Existing,300+km/hr,300,(LINESTRING Z (120.18048509 30.33301164900007 ...
4,5,京港通道,jīng gǎng tōng dào,Existing,300+km/hr,300,(LINESTRING Z (113.2928800510001 22.9317548520...
10,11,盘营联络线,pán yíng lián lào xiàn,Existing,300+km/hr,300,(LINESTRING Z (122.578932277 40.86892311900004...
24,25,台灣高鐵,tái wān gāo tiě,Existing,300+km/hr,300,LINESTRING Z (120.307548868 22.68714827100007 ...


#### Get points from routes

In [52]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        else:
            g = [m for m in route.geometry]
            for j in range(0,len(g)):
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
                    tmp = (sec_points[k][0],sec_points[k][1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

8046 points


In [53]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),163):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points),'points')

50 points


#### Create working dataset

In [54]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,120.20826146800005_30.293415948000074,120.208261,30.293416
1,121.53196055800004_29.93712074200005,121.531961,29.937121
2,120.00065641200001_30.981595850000073,120.000656,30.981596
3,113.7733801930001_22.851714791000063,113.773380,22.851715
4,115.7645844220001_39.05651377700008,115.764584,39.056514


In [190]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [55]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))

# Set map centerpoint
coords = marker_points[int((len(marker_points)/2))]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 7
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [191]:
'''
Get satellite preview for image
'''

row = 1

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [192]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+output_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 19,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 